In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://arch

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-30 01:24:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.92MB/s    in 0.2s    

2021-08-30 01:24:14 (5.92 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Create df to show total_votes count is >= 20
total_votes_df = df.select(['product_id', 'product_title', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'star_rating', 'review_body'])
high_total_votes_df = total_votes_df.filter('total_votes >= 20')
high_total_votes_df.show()

+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|product_id|       product_title|helpful_votes|total_votes|vine|verified_purchase|star_rating|         review_body|
+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|B00QZLVCU0|Valve - DOTA 2 St...|           21|         34|   N|                N|          1|Who pays 4 dollar...|
|B00RHI62GY|ONE PIECE Pirate ...|           21|         35|   N|                Y|          1|Yesterday i recei...|
|B00W435BU0|       Madden NFL 16|          147|        175|   N|                Y|          1|This is an absolu...|
|B00PV515DU|Sony Borderlands ...|           14|         31|   N|                Y|          1|I ordered this it...|
|B00WJ0Z194|Infinity 3.0 Star...|           55|         60|   N|                N|          2|While Infinity 3....|
|B00W435C0Y|       Madden NFL 16|           51|         65|   N|        

In [7]:
high_total_votes_df.count()

65379

In [8]:
# Filter and create a new df to retrieve all the rows where helpful_votes / total_votes >=50%
high_percent_votes_df = high_total_votes_df.filter('helpful_votes / total_votes >= 0.5')
high_percent_votes_df.show()

+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|product_id|       product_title|helpful_votes|total_votes|vine|verified_purchase|star_rating|         review_body|
+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|B00QZLVCU0|Valve - DOTA 2 St...|           21|         34|   N|                N|          1|Who pays 4 dollar...|
|B00RHI62GY|ONE PIECE Pirate ...|           21|         35|   N|                Y|          1|Yesterday i recei...|
|B00W435BU0|       Madden NFL 16|          147|        175|   N|                Y|          1|This is an absolu...|
|B00WJ0Z194|Infinity 3.0 Star...|           55|         60|   N|                N|          2|While Infinity 3....|
|B00W435C0Y|       Madden NFL 16|           51|         65|   N|                Y|          1|I've owned a vers...|
|B00WJ0Z10S|Infinity 3.0 Star...|           31|         36|   N|        

In [9]:
# Filter and create a new df to retreive all the rows where a review was written as part of Vine
high_percent_votes_withVine_df = high_total_votes_df.filter('vine = True')
high_percent_votes_withVine_df.show()

+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|product_id|       product_title|helpful_votes|total_votes|vine|verified_purchase|star_rating|         review_body|
+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|B00YXO5UXG|Turtle Beach - Ea...|           56|         63|   Y|                N|          5|Not every situati...|
|B00XO041RQ|PDP AG7 True Wire...|           23|         23|   Y|                N|          3|[[VIDEOID:a9ebb8e...|
|B00YXO5X4M|Turtle Beach - Ea...|           19|         24|   Y|                N|          5|[[VIDEOID:5a58574...|
|B00U33Q940|NVIDIA SHIELD - 4...|            0|         25|   Y|                N|          1|I am really glad ...|
|B00U33Q940|NVIDIA SHIELD - 4...|            2|         28|   Y|                N|          3|I got this in tod...|
|B00P45U43O|Antec X-1 Cooler ...|           22|         26|   Y|        

In [11]:
high_percent_votes_withVine_df.count()

104

In [12]:
# Filter and create a new df to retreive all the rows where a review was written as part of Vine
high_percent_votes_withVine_df = high_total_votes_df.filter('vine = True')
high_percent_votes_withVine_df.show()

+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|product_id|       product_title|helpful_votes|total_votes|vine|verified_purchase|star_rating|         review_body|
+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|B00YXO5UXG|Turtle Beach - Ea...|           56|         63|   Y|                N|          5|Not every situati...|
|B00XO041RQ|PDP AG7 True Wire...|           23|         23|   Y|                N|          3|[[VIDEOID:a9ebb8e...|
|B00YXO5X4M|Turtle Beach - Ea...|           19|         24|   Y|                N|          5|[[VIDEOID:5a58574...|
|B00U33Q940|NVIDIA SHIELD - 4...|            0|         25|   Y|                N|          1|I am really glad ...|
|B00U33Q940|NVIDIA SHIELD - 4...|            2|         28|   Y|                N|          3|I got this in tod...|
|B00P45U43O|Antec X-1 Cooler ...|           22|         26|   Y|        

In [13]:
# Filter and create a new df to retreive all the rows where a review was written as part of Vine
high_percent_votes_noVine_df = high_total_votes_df.filter('vine = False')
high_percent_votes_noVine_df.show()

+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|product_id|       product_title|helpful_votes|total_votes|vine|verified_purchase|star_rating|         review_body|
+----------+--------------------+-------------+-----------+----+-----------------+-----------+--------------------+
|B00QZLVCU0|Valve - DOTA 2 St...|           21|         34|   N|                N|          1|Who pays 4 dollar...|
|B00RHI62GY|ONE PIECE Pirate ...|           21|         35|   N|                Y|          1|Yesterday i recei...|
|B00W435BU0|       Madden NFL 16|          147|        175|   N|                Y|          1|This is an absolu...|
|B00PV515DU|Sony Borderlands ...|           14|         31|   N|                Y|          1|I ordered this it...|
|B00WJ0Z194|Infinity 3.0 Star...|           55|         60|   N|                N|          2|While Infinity 3....|
|B00W435C0Y|       Madden NFL 16|           51|         65|   N|        

In [14]:
# Determine the total number of reviews
high_total_votes_df.count()

65379

In [15]:
# Determine the total number of 5-star reviews
five_star_df = high_total_votes_df.filter(df['star_rating']== 5)
five_star_df.count()


20487

In [16]:
# Determine the % of 5-star reviews for paid
five_star_paid_df = five_star_df.filter(five_star_df['verified_purchase']== 'Y')
five_star_paid_df.count()

4704

In [17]:
# Percentage of 5-star paid reviews vs all five_star reviews
five_star_paid_df.count() / five_star_df.count()

0.22960902035437106

In [18]:
# Determine the % of 5-star reviews for unpaid
five_star_unpaid_df = five_star_df.filter(five_star_df['verified_purchase']== 'N')
five_star_unpaid_df.count()

15783

In [19]:
# Percentage of 5-star unpaid reviews vs all five_star reviews
five_star_unpaid_df.count() / five_star_df.count()

0.770390979645629